In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


import pandas as pd


In [17]:
pd.set_option('display.max_columns', None)


In [7]:
# Inicializa o Firebase Admin, se ainda não estiver inicializado
if not firebase_admin._apps:
    cred = credentials.Certificate('firebase-key.json')
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [11]:
def listar_colecoes_e_documentos(head = True):
    colecoes = db.collections()  # Lista todas as coleções
    for colecao in colecoes:
        print(f'Nome da Coleção: {colecao.id}')
        documentos = colecao.stream()
        for idx, documento in enumerate(documentos):
            print(f' - Documento ID: {documento.id}, Dados: {documento.to_dict()}')
            if (head == True) and (idx == 9):
                break

# Chamada da função
listar_colecoes_e_documentos()

Nome da Coleção: CNPJ
 - Documento ID: 03812120000181_false, Dados: {'CNPJ__c': '03.812.120/0001-81', 'Email': 'borracheiro@teste.com'}
 - Documento ID: 03974743000150_false, Dados: {'CNPJ__c': '03.974.743/0001-50', 'Email': 'tester@teste.com'}
 - Documento ID: 04522171000131_false, Dados: {'CNPJ__c': '04.522.171/0001-31', 'Email': 'caroool@carol.com'}
 - Documento ID: 11934656000106_false, Dados: {'CNPJ__c': '11.934.656/0001-06', 'Email': 'teste1@teste.com'}
 - Documento ID: 16046110000124_false, Dados: {'CNPJ__c': '16.046.110/0001-24', 'Email': 'teste92@teste.com'}
 - Documento ID: 30449012000169_false, Dados: {'CNPJ__c': '30.449.012/0001-69', 'Email': 'n@teste.com'}
 - Documento ID: 35118924000171_false, Dados: {'CNPJ__c': '35.118.924/0001-71', 'Email': 'novoos@teste.com'}
 - Documento ID: 35895109000119_false, Dados: {'CNPJ__c': '35.895.109/0001-19', 'Email': 'nooo@teste.com'}
 - Documento ID: 37668182000183_false, Dados: {'CNPJ__c': '37.668.182/0001-83', 'Email': 'teste@jose.com'}

In [12]:
def colecao_para_dataframe(nome_colecao):
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()

    # Criar uma lista para armazenar os dados dos documentos
    documentos_lista = []

    # Iterar sobre cada documento e adicionar ao dicionário, em seguida, adicionar à lista
    for doc in documentos:
        doc_dict = doc.to_dict()
        doc_dict['document_id'] = doc.id  # Incluir o ID do documento, se necessário
        documentos_lista.append(doc_dict)

    # Converter a lista de dicionários em um DataFrame
    if documentos_lista:
        dataframe = pd.DataFrame(documentos_lista)
    else:
        dataframe = pd.DataFrame()  # Retorna um DataFrame vazio se não houver documentos

    return dataframe


In [18]:
# Exemplo de uso
nome_colecao = 'usuarios'
df = colecao_para_dataframe(nome_colecao)
display(df)

,bairro,cep,uid,data_nascimento,longitude,endereco,pushToken,complemento,apiPaymentCode,nome_completo,online,liner,betaTest,isDeviceLoggedIn,estado,latitude,telefone,cidade,numero,email,cpf,document_id,deslogar_hash,deslogar,outraCategoria,razao_social,status,categorias,cnpj
0,Jardim Fraternidade,05870-030,sOEX7DhnwAhUCpMwsbNMhhgJ1A63,01/01/2000,-46.770504,Rua Andréa Solario,,,NaN,Teste5,False,True,True,False,SP,-23.679828,(11) 9999-9999,São Paulo,111,12ewdsa@sdfsd.com,841.903.430-48,12ewdsa@sdfsd.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vale dos Lagos,41256-300,2Q1qhRgthLT4ZQUPvmDLHDUF55t2,20/03/1999,-38.420261,Alameda Mata Atlântica II,None,,NaN,Aaasssa,False,True,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,aaaaabbbbb@jose.com,576.902.630-89,aaaaabbbbb@jose.com,0951b3b36f9e6041dbbc912aae7f55af,False,NaN,NaN,NaN,NaN,NaN
2,Vale dos Lagos,41256-300,RodBeP0uKdd17Q8oegcaHiZeMnB2,20/03/1998,-38.420261,Alameda Mata Atlântica II,None,,NaN,AAbabelado,False,True,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,aabbabelado@jose.com,445.579.440-94,aabbabelado@jose.com,050b9e806ad71c9714116d11c2d417fc,False,NaN,NaN,NaN,NaN,NaN
3,Vale dos Lagos,41256-300,zmGH8GihTeeJk5hBxxhSvdVTe1a2,20/03/1975,-38.420261,Alameda Mata Atlântica II,None,,NaN,Keu,False,True,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,adora@jose.com,172.834.783-10,adora@jose.com,f864b38539e0fb4133fc1cbe2b78637e,False,NaN,NaN,NaN,NaN,NaN
4,Chácara Santo Antônio (Zona Sul),04717-030,W6XlbN7659XlonVTGE3jPuiGAhj2,01/01/2000,-46.697251,Rua Gaetano Antônio Marmo,NaN,,NaN,Teste novo cadastro,NaN,True,True,False,SP,-23.636207,(11) 99999-9999,São Paulo,123,ajah@jdjd.com,448.493.306-30,ajah@jdjd.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Vila Gomes Cardim,03320-000,92jYlKup3ndihKxHQbAs42QEpa43,20/05/1888,-46.573645,Rua Emílio Mallet,None,,NaN,Marcela,False,True,True,False,SP,-23.548063,(85) 66666-6666,São Paulo,5,testeline@line.com,054.712.240-35,testeline@line.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Vila Gomes Cardim,03320-000,u0vm5w47sBZVJUYfSxvrbSpoSka2,NaN,-46.573002,Rua Emílio Mallet,ExponentPushToken[QGE66UIUu6OS_SQnpXnNic],,NaN,Carol,False,False,True,False,SP,-23.547844,(15) 45548-4848,São Paulo,77,tester@teste.com,179.200.850-39,tester@teste.com,NaN,NaN,,Salão do bairro,Ativo,[Salão de cabeleireiro/ Barbearia],03.974.743/0001-50
82,Vale dos Lagos,41256-300,2U9OuGm8QjWonIORa2SXCAJrrrG3,NaN,-38.420261,Alameda Mata Atlântica II,None,,75982968.0,Cleiton,False,False,True,False,BA,-12.930147,(71) 99314-5125,Salvador,106,testwork@jose.jose,032.938.990-48,testwork@jose.jose,e3af150ecc1f993e35860b6cfedb48dc,False,,Teste Work informática,Ativo,"[Saúde e bem estar, Bolos e doces]",74.958.598/0001-62
83,Vila Gomes Cardim,03320-000,Va9opjW3BjQcbZ69glGAhQ5cYwz1,NaN,-46.573695,Rua Emílio Mallet,None,,NaN,N,False,False,True,False,SP,-23.548181,(81) 54548-4848,São Paulo,8,yyy@teste.com,404.274.790-61,yyy@teste.com,28813873230f39e131694bcbabcdaf09,False,,N,Ativo,[Odontológica],


In [ ]:
def criar_documento(nome_colecao, dados, doc_id=None):
    """
    Cria um novo documento em uma coleção especificada no Firestore.

    :param nome_colecao: String que especifica o nome da coleção.
    :param dados: Dicionário contendo os dados a serem inseridos no novo documento.
    :param doc_id: String opcional que especifica o ID do documento. Se não for fornecido,
                   o Firestore gera um ID único automaticamente.
    """
    colecao_ref = db.collection(nome_colecao)

    if doc_id:
        doc_ref = colecao_ref.document(doc_id)
        doc_ref.set(dados)
        print(f'Documento criado com ID: {doc_id}')
    else:
        doc_ref = colecao_ref.add(dados)
        print(f'Documento criado com ID gerado pelo Firestore: {doc_ref[1].id}')

def ler_todos_documentos(nome_colecao):
    """
    Lê e imprime todos os documentos de uma coleção especificada no Firestore.

    :param nome_colecao: String que especifica o nome da coleção a ser lida.
    """
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()
    
    for doc in documentos:
        print(f'{doc.id} => {doc.to_dict()}')

def ler_documento(nome_colecao, doc_id):
    """
    Lê e imprime um documento específico de uma coleção pelo seu ID.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser lido.
    """
    doc_ref = db.collection(nome_colecao).document(doc_id)
    doc = doc_ref.get()
    if doc.exists:
        print(f'Documento encontrado: {doc.id} => {doc.to_dict()}')
    else:
        print('Documento não encontrado')

def atualizar_documento(nome_colecao, doc_id, novos_dados):
    """
    Atualiza um documento específico em uma coleção com novos dados.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser atualizado.
    :param novos_dados: Dicionário contendo os novos dados para atualizar o documento.
    """
    doc_ref = db.collection(nome_colecao).document(doc_id)
    doc_ref.update(novos_dados)
    print(f'Documento {doc_id} atualizado com sucesso.')

def deletar_documento(nome_colecao, doc_id):
    """
    Deleta um documento específico de uma coleção pelo seu ID.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser deletado.
    """
    db.collection(nome_colecao).document(doc_id).delete()
    print(f'Documento {doc_id} deletado com sucesso.')

